In [7]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [9]:
def C45(data,originaldata,features,target_attribute="Label",parent_node = None):
    
    if len(np.unique(data[target_attribute])) <= 1:
        return np.unique(data[target_attribute])[0]
    
    elif len(data)==0:
        return np.unique(originaldata[target_attribute])[np.argmax(np.unique(originaldata[target_attribute],return_counts=True)[1])]
    
    elif len(features) ==0:
        return parent_node
    
    elif [GainRatio(data,feature,target_attribute) for feature in features] == 0:
        return parent_node
    
    else:
        parent_node = np.unique(data[target_attribute])[np.argmax(np.unique(data[target_attribute],return_counts=True)[1])]
        
        item_values = [GainRatio(data,feature,target_attribute) for feature in features] 
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        tree = {best_feature:{}}
        
        new_features_set = []
        for i in features:
            if i!=best_feature:
                new_features_set.append(i)
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            
            subtree = C45(sub_data,originaldata,new_features_set,target_attribute,parent_node)
            
            tree[best_feature][value] = subtree
            
        return(tree)
 
def GainRatio(data, splitAttribute, targetFeature):
    info_gain = InfoGain(data, splitAttribute, targetFeature)
    vals, counts = np.unique(data[splitAttribute], return_counts=True)
    counts = list(counts)
    split_info_values = []
    for i in range(len(counts)):
        split_info_values.append(-(counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)))
    gain_ratio = info_gain/np.sum(split_info_values)
    return gain_ratio

def entropy(features):
    labels, counts = np.unique(features, return_counts=True)
    counts = list(counts)
    entropy_values = []
    for i in range(len(labels)):
        val = (-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts))
        entropy_values.append(val)
    return np.sum(entropy_values)

def DecisionTreeClassifier(query,tree, Label='Win'):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return Label
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return DecisionTreeClassifier(query,result)
            else:
                return result

def InfoGain(data, splitAttribute, targetFeature):
    entropy_pre = entropy(data[targetFeature])
    vals, counts = np.unique(data[splitAttribute], return_counts=True)
    counts = list(counts)
    values_post_entropy = []
    for i in range(len(vals)):
        feature_target = data.where(data[splitAttribute]==vals[i]).dropna()[targetFeature]
        i_entropy = entropy(feature_target)
        values_post_entropy.append((counts[i]/np.sum(counts))*i_entropy) 
    entropy_post = np.sum(values_post_entropy)
    
    info_gain = entropy_pre - entropy_post
    return info_gain

train = pd.read_csv('C:/Users/Harsh/Desktop/task5/train.csv')

test = pd.read_csv('C:/Users/Harsh/Desktop/task5/test.csv')
true_label = list(test["Label"])
features= list(train)[1:-1]

Label = list(train)[-1]

tree_C45 = C45(train,train,features[:],Label)

print(tree_C45)

queries = test.iloc[:,1:-1].to_dict(orient = "records")

predicted_C45 = []
for i in range(len(test)):
    predicted_C45.append(DecisionTreeClassifier(queries[i],tree_C45,1.0))


print("true_label:")
print(true_label)


print("predicted_C45:")
print(predicted_C45)

print("C45 Classification Model:")
print("Accuracy: ", metrics.accuracy_score(true_label, predicted_C45))
print(classification_report(true_label, predicted_C45))
print()

C:\Users\harsh\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in double_scalars


{'In_out': {'In': {'Is_home_away': {'Home': {'Media': {'NBC': 'Lose'}}, 'Away': 'Lose'}}, 'Out': {'Media': {'ABC': {'Is_home_away': {'Away': 'Win'}}, 'CBS': 'Lose', 'ESPN': 'Win', 'NBC': {'Is_home_away': {'Home': 'Win', 'Away': 'Win'}}}}}}
true_label:
['Win', 'Lose', 'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Win', 'Lose', 'Win', 'Lose']
predicted_C45:
['Win', 'Lose', 'Win', 'Win', 'Win', 'Win', 'Lose', 'Win', 'Win', 'Lose', 'Win', 'Lose']
C45 Classification Model:
Accuracy:  0.9166666666666666
              precision    recall  f1-score   support

        Lose       0.75      1.00      0.86         3
         Win       1.00      0.89      0.94         9

    accuracy                           0.92        12
   macro avg       0.88      0.94      0.90        12
weighted avg       0.94      0.92      0.92        12


